In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

# Setup Edge WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

# Automatically download and use the correct Edge WebDriver
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)

# Test if Edge opens
driver.get("https://www.google.com")
print("✅ Edge WebDriver is working!")
driver.quit()


✅ Edge WebDriver is working!


In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import csv

# Lazada product URL (replace with actual product URL)
url = "https://www.lazada.com.my/products/ugreen-braided-mfi-lightning-usb-a-to-lightning-cable-iphone-fast-charging-cable-for-iphoneipad-i12749949-s10763212739.html"

# Setup Edge WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

# Start WebDriver
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
driver.get(url)
wait = WebDriverWait(driver, 10)

# ✅ Step 1: Scroll down to load reviews
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)  # Give time for reviews to load

# ✅ Step 2: Check if the review section exists
try:
    review_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "pdp-mod-review")))
    print("✅ Found review section!")
except:
    print("⚠️ No review section found. Exiting script.")
    driver.quit()
    exit()

# ✅ Step 3: Scrape reviews from 5 pages
review_data = []
for page in range(5):  # Loop through first 5 pages
    try:
        reviews_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "mod-reviews")))
        reviews = reviews_container.find_elements(By.CLASS_NAME, "item")
        print(f"✅ Found {len(reviews)} reviews on page {page + 1}!")

        for review in reviews:
            try:
                # Extract review date from 'title right' class inside 'top' section
                try:
                    date_element = review.find_element(By.XPATH, './/div[contains(@class, "top")]//span[contains(@class, "title right")]')
                    review_date = date_element.text.strip()
                except:
                    review_date = "Unknown Date"

                # Extract name from span (no class)
                try:
                    name_element = review.find_element(By.XPATH, './/div[contains(@class, "middle")]//span')
                    name = name_element.text.strip()
                except:
                    name = "Unknown"

                # Extract review text from 'content' class
                try:
                    comment = review.find_element(By.CLASS_NAME, "content").text.strip()
                except:
                    comment = "No review text"

                review_data.append([name, review_date, comment])
                print(f"✅ Scraped: {name} - {review_date} - {comment[:50]}...")
            except Exception as e:
                print(f"⚠️ Skipping a review due to an error: {e}")

        # ✅ Step 4: Click the 'Next' button for pagination
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "next-btn next-btn-normal next-btn-medium next-pagination-item next")]')))
            driver.execute_script("arguments[0].click();", next_button)
            print(f"➡️ Moving to page {page + 2}...")
            time.sleep(3)  # Wait for new reviews to load
        except:
            print("⚠️ No 'Next' button found or last page reached.")
            break

    except:
        print("⚠️ Could not find the reviews container on page {page + 1}. Exiting script.")
        break

# ✅ Step 5: Save to CSV if there are reviews
if review_data:
    with open("lazada_reviews.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Reviewer Name", "Review Date", "Review"])
        writer.writerows(review_data)
    print("✅ Scraping complete. Data saved to 'lazada_reviews.csv'.")
else:
    print("⚠️ No reviews found.")

driver.quit()


✅ Found review section!
✅ Found 5 reviews on page 1!
✅ Scraped: W***. - 3 weeks ago - Perfect for iPhone and iPad users, High-quality br...
✅ Scraped: Harwant S. - 26 Jan 2025 - Shipment was fast. Length is just perfect for use ...
✅ Scraped: Chong F. - 1 week ago - 💲Price:mfi cable expensive 
 🛡Durability:normal ca...
✅ Scraped: T***. - 1 week ago - Great value for money, Reliable and long-lasting, ...
✅ Scraped: Teoh W. - 2 weeks ago - Fast data transfer speed, Great value for money, C...
➡️ Moving to page 2...
✅ Found 5 reviews on page 2!
✅ Scraped: W***. - 3 weeks ago - Perfect for iPhone and iPad users, High-quality br...
✅ Scraped: Harwant S. - 26 Jan 2025 - Shipment was fast. Length is just perfect for use ...
✅ Scraped: Chong F. - 1 week ago - 💲Price:mfi cable expensive 
 🛡Durability:normal ca...
✅ Scraped: T***. - 1 week ago - Great value for money, Reliable and long-lasting, ...
✅ Scraped: Teoh W. - 2 weeks ago - Fast data transfer speed, Great value for money, C...
➡️ Moving